In [1]:
#Bibliotecas p/ importação 
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.metrics import confusion_matrix

sc = StandardScaler()# Método de Estandardização por Escalonamento

Using TensorFlow backend.


In [2]:
#Leitura dos dados de entrada X e dos rótulos de estabilidade y
X = pd.read_csv('C:\\Users\\Pablo\\Desktop\\tfc\\Simulações\\Data.csv', index_col = 0)
y = pd.read_csv('C:\\Users\\Pablo\\Desktop\\tfc\\Simulações\\y.csv', usecols = [1])

#Separação das variáveis - 86 variáveis ao todo
Dmax = X.iloc[:,:7]
Dmin = X.iloc[:,7:14]
D = X.iloc[:,:14]
PGmean = X.iloc[:,14:22]
PGmax = X.iloc[:,22:30]
PGmin = X.iloc[:,30:38]
PG = X.iloc[:,14:38]
QGmean = X.iloc[:,38:46]
QGmax = X.iloc[:,46:54]
QGmin = X.iloc[:,54:62]
QG = X.iloc[:,38:62]
VMmean = X.iloc[:,62:70]
VMmax = X.iloc[:,70:78]
VMmin = X.iloc[:,78:86]
VM = X.iloc[:,62:86]
    
parametros = {'Kernel':'linear', #Parâmetro p/ SVM - Tipo de kernel
             'Funcao perda':'binary_crossentropy',#Parâmetro p/ RNA - Função de pontuação de otimização
             'Otimizador':'SGD',#Parâmetro p/ RNA - Forma de cálculo de descida do gradiente do treinamento
             'Ativacao camadas':'relu',#Parâmetro p/ RNA - Função de ativação das camadas de entrada e escondidas
             'Ativacao saida':'sigmoid',#Parâmetro p/ RNA - Função de ativação da camada de saída
             }

conjuntos = {'D':D, 'PG':PG, 'QG':QG, 'VM':VM, 'Dmin':Dmin, 'Dmax':Dmax, 
             'PGmin':PGmin, 'PGmax':PGmax, 'QGmin':QGmin, 'QGmax':QGmax, 
             'VMmin':VMmin, 'VMmax':VMmax} #Dicionário com cada conjunto associado a um índice
    
index = list()
for x in conjuntos: index.append(x) #Lista de índices com os conjuntos

In [3]:
#Função para Treinamento e Teste de uma SVM
#Ao dicionário 'conjuntos' atribui-se os 12 conjuntos de dados definidos 
#A divisão dos dados é sempre aleatoria, portanto para cada execução da função é esperado resultados diferentes
#As saídas da função são as pontuações de precisão, uma de treino e outra de teste, obtidos na execução

def svm(conjuntos, y, C, kernel):
    precisao_treino = list()
    precisao_teste = list()
    for conjunto in conjuntos: #Para cada índice do dicionário 'conjuntos',
        X = conjuntos[str(conjunto)] # a variável X recebe o índice
        X_treino, X_teste, y_treino, y_teste = train_test_split(sc.fit_transform(abs(X.values)), y.values.ravel(),
                                                            test_size=0.3, shuffle = True) #Separação dos dados
        clf_svm = SVC(C=C, kernel=kernel) #É de interesse apenas C e kernel, com o resto dos parâmetros em default
        clf_svm.fit(X_treino, y_treino) #O Classificador se ajusta aos dados de treinamento
        precisao_treino.append(clf_svm.score(X_treino, y_treino)) #Pontuação de precisão de treino
        precisao_teste.append(clf_svm.score(X_teste, y_teste)) #Pontuação de precisão de teste
    return precisao_treino, precisao_teste

In [4]:
#Função p/ treinamento e teste de RNA 
#caso 0 - Apenas camadas de entrada e saída (2 layers)
#caso 1 - Uma única camada entre camadas de entrada e saída (3 layers)
#caso 2 - Quatro camadas entre camadas de entrada e saída (6 layers)
#A saída da função é uma única tabela resumo dos resultados obtidos na execução
def rna(conjuntos, index, caso, parametros):
    resultado = pd.DataFrame()
    perda = parametros['Funcao perda']
    otimizador = parametros['Otimizador']
    fatv_camadas = parametros['Ativacao camadas']
    fatv_camada_saida = parametros['Ativacao saida']
    
    for conjunto in conjuntos: #Para cada índice do dicionário 'conjuntos',
        X = conjuntos[str(conjunto)] # a variável X recebe o índice
        X_treino, X_teste, y_treino, y_teste = train_test_split(sc.fit_transform(abs(X.values)), y.values.ravel(),
                                                            test_size=0.3, shuffle = True) #Separação dos dados
        #Modelagem de camadas de neurônios
        modelo = Sequential()
        
        #Camada de Entrada  -> dimensão da entrada = número de variáveis
        modelo.add(Dense(64, input_dim=X_treino.shape[1], activation=fatv_camadas))
        
        if caso == 1:
            modelo.add(Dense(32, input_dim=64, activation=fatv_camadas))
        elif caso == 2:
            modelo.add(Dense(32, input_dim=64, activation=fatv_camadas))
            modelo.add(Dense(16, input_dim=32, activation=fatv_camadas))
            modelo.add(Dense(8, input_dim=16, activation=fatv_camadas))
            modelo.add(Dense(4, input_dim=8, activation=fatv_camadas))
            
        #Camada de Saída
        modelo.add(Dense(1, activation=fatv_camada_saida))
        
        #Compila o modelo - baseia-se no dicionário 'parametros'
        modelo.compile(loss=perda, 
                       optimizer=otimizador,
                       metrics=['accuracy'])
        
        #17.5% dos dados de treino p/ CV - 25% dos dados do conjunto
        #Batch size - número de amostras processadas antes de atualizar o modelo da RNA
        batch_size = 128
        
        #Steps per epoch - número de iterações de batch antes de terminar uma época = nº amostras/tamanho do batch
        steps_per_epoch = int(X_treino.shape[0]/batch_size)
        
        #Ajuste do modelo
        clf_rna = modelo.fit(X_treino,y_treino, validation_split=0.175, batch_size=None, epochs=30, 
                             validation_steps = steps_per_epoch*2, steps_per_epoch=steps_per_epoch)
        #Aavaliação do Modelo
        eval_treino=modelo.evaluate(X_treino, y_treino)
        eval_teste=modelo.evaluate(X_teste, y_teste)
        
        #Predição com os dados de teste
        y_pred = modelo.predict(X_teste)
        
        #Classificação p/ construção de matriz de confusão
        #y_pred>0.5 -> True -> Classificado corretamente; y_pred>0.5 -> False -> Classificado incorretamente
        y_pred = (y_pred>0.5)    
        cm = confusion_matrix(y_teste, y_pred)
        
        #Construção da tabela de saída
        resultado = resultado.append({'Teste - Acertos p/ Estável':cm[0][0],
                         'Teste - Erros p/ Estável':cm[0][1],
                         'Teste - Acertos p/ Instável':cm[1][1],
                         'Teste - Erros p/ Instável':cm[1][0],
                         'Precisão de Treino (%)':eval_treino[1]*100,
                         'Precisão de Teste (%)':eval_teste[1]*100}, ignore_index=True)
    resultado.set_index(np.array(index), inplace=True)
        
    return resultado

In [5]:
#Define-se a construção das saídas do programa

#A função 'saida_rna()' tem como saída 3 tabelas, executando a função os três casos de número de camadas de neurônios
def saida_rna(parametros, X, y):
    rna_2camadas = rna(conjuntos, index, 0, parametros)
    
    rna_3camadas = rna(conjuntos, index, 1, parametros)
    
    rna_6camadas = rna(conjuntos, index, 2, parametros)
    
    return rna_2camadas, rna_3camadas, rna_6camadas
#===============================================================================================================================
#A função 'saida_svm()' tem como saída duas tabelas constrúidas a partir da função 'svm()' para quatro valores de C
def saida_svm(parametros, X, y):
    svm_resultado_teste = pd.DataFrame({'Precisão de Teste (%), C=1': svm(conjuntos, y, 1, parametros['Kernel'])[1],
                                        'Precisão de Teste (%), C=0.1': svm(conjuntos, y, 0.1, parametros['Kernel'])[1],
                                        'Precisão de Teste (%), C=0.01': svm(conjuntos, y, 0.01, parametros['Kernel'])[1],
                                        'Precisão de Teste (%), C=0.001': svm(conjuntos, y, 0.001, parametros['Kernel'])[1]},
                                       index = index) #Construção da tabela de resultados da SVM para teste

    svm_resultado_treino = pd.DataFrame({'Precisão de Treino (%), C=1': svm(conjuntos, y, 1, parametros['Kernel'])[0],
                                     'Precisão de Treino (%), C=0.1': svm(conjuntos, y, 0.1, parametros['Kernel'])[0],
                                     'Precisão de Treino (%), C=0.01': svm(conjuntos, y, 0.01, parametros['Kernel'])[0],
                                     'Precisão de Treino (%), C=0.001': svm(conjuntos, y, 0.001, parametros['Kernel'])[0]},
                                        index = index) #Construção da tabela de resultados da SVM para treino
    #p/ resultado percentual
    svm_resultado_treino = svm_resultado_treino.apply(lambda x: x*100)
    svm_resultado_teste = svm_resultado_teste.apply(lambda x: x*100)
    
    return svm_resultado_treino, svm_resultado_teste

In [6]:
Treino_SVM = saida_svm(parametros, X, y)[0]
Teste_SVM = saida_svm(parametros, X, y)[1]

In [8]:
Resultado_SVM = saida_rna(parametros, X, y)

W0815 16:40:29.715508 11572 deprecation_wrapper.py:119] From C:\Users\Pablo\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0815 16:40:30.210913 11572 deprecation_wrapper.py:119] From C:\Users\Pablo\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0815 16:40:30.319669 11572 deprecation_wrapper.py:119] From C:\Users\Pablo\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0815 16:40:30.458595 11572 deprecation_wrapper.py:119] From C:\Users\Pablo\Anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Train on 2171 samples, validate on 461 samples
Epoch 1/30
20/20 [==============================] - 1s 50ms/step - loss: 0.5635 - acc: 0.6863 - val_loss: 0.5090 - val_acc: 0.9523
Epoch 2/30
20/20 [==============================] - 0s 4ms/step - loss: 0.4740 - acc: 0.9546 - val_loss: 0.4355 - val_acc: 0.9610
Epoch 3/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4092 - acc: 0.9633 - val_loss: 0.3805 - val_acc: 0.9718
Epoch 4/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3600 - acc: 0.9748 - val_loss: 0.3376 - val_acc: 0.9892
Epoch 5/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3211 - acc: 0.9832 - val_loss: 0.3029 - val_acc: 0.9892
Epoch 6/30
20/20 [==============================] - 0s 3ms/step - loss: 0.2893 - acc: 0.9880 - val_loss: 0.2742 - val_acc: 0.9957
Epoch 7/30
20/20 [==============================] - 0s 3ms/step - loss: 0.2628 - acc: 0.9912 - val_loss: 0.2498 - val_acc: 0.9957
Epoch 8/30
20/20 [========================

20/20 [==============================] - 0s 13ms/step - loss: 0.6314 - acc: 0.5582 - val_loss: 0.5815 - val_acc: 0.7809
Epoch 2/30
20/20 [==============================] - 0s 3ms/step - loss: 0.5360 - acc: 0.8685 - val_loss: 0.5057 - val_acc: 0.8633
Epoch 3/30
20/20 [==============================] - 0s 4ms/step - loss: 0.4722 - acc: 0.9083 - val_loss: 0.4531 - val_acc: 0.8980
Epoch 4/30
20/20 [==============================] - 0s 5ms/step - loss: 0.4267 - acc: 0.9196 - val_loss: 0.4145 - val_acc: 0.8937
Epoch 5/30
20/20 [==============================] - 0s 6ms/step - loss: 0.3927 - acc: 0.9193 - val_loss: 0.3851 - val_acc: 0.8937
Epoch 6/30
20/20 [==============================] - 0s 4ms/step - loss: 0.3661 - acc: 0.9169 - val_loss: 0.3616 - val_acc: 0.8894
Epoch 7/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3448 - acc: 0.9103 - val_loss: 0.3424 - val_acc: 0.8894
Epoch 8/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3272 - acc: 0.9061 - val_l

20/20 [==============================] - 0s 19ms/step - loss: 0.6501 - acc: 0.6004 - val_loss: 0.6100 - val_acc: 0.7679
Epoch 2/30
20/20 [==============================] - 0s 3ms/step - loss: 0.5859 - acc: 0.7824 - val_loss: 0.5501 - val_acc: 0.8785
Epoch 3/30
20/20 [==============================] - 0s 3ms/step - loss: 0.5347 - acc: 0.8548 - val_loss: 0.5019 - val_acc: 0.8959
Epoch 4/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4933 - acc: 0.8677 - val_loss: 0.4625 - val_acc: 0.9089
Epoch 5/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4593 - acc: 0.8750 - val_loss: 0.4297 - val_acc: 0.9132
Epoch 6/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4308 - acc: 0.8837 - val_loss: 0.4021 - val_acc: 0.9154
Epoch 7/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4067 - acc: 0.8921 - val_loss: 0.3785 - val_acc: 0.9176
Epoch 8/30
20/20 [==============================] - 0s 3ms/step - loss: 0.3861 - acc: 0.8965 - val_l

20/20 [==============================] - 0s 22ms/step - loss: 0.6078 - acc: 0.7156 - val_loss: 0.5759 - val_acc: 0.7484
Epoch 2/30
20/20 [==============================] - 0s 3ms/step - loss: 0.5628 - acc: 0.7420 - val_loss: 0.5405 - val_acc: 0.7462
Epoch 3/30
20/20 [==============================] - 0s 3ms/step - loss: 0.5327 - acc: 0.7530 - val_loss: 0.5158 - val_acc: 0.7462
Epoch 4/30
20/20 [==============================] - 0s 3ms/step - loss: 0.5111 - acc: 0.7568 - val_loss: 0.4978 - val_acc: 0.7419
Epoch 5/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4950 - acc: 0.7567 - val_loss: 0.4842 - val_acc: 0.7505
Epoch 6/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4825 - acc: 0.7596 - val_loss: 0.4735 - val_acc: 0.7744
Epoch 7/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4725 - acc: 0.7693 - val_loss: 0.4648 - val_acc: 0.7787
Epoch 8/30
20/20 [==============================] - 0s 3ms/step - loss: 0.4642 - acc: 0.7796 - val_l

...

20/20 [==============================] - 1s 37ms/step - loss: 0.7291 - acc: 0.4675 - val_loss: 0.6847 - val_acc: 0.5857
Epoch 2/30
20/20 [==============================] - 0s 4ms/step - loss: 0.6540 - acc: 0.6931 - val_loss: 0.6196 - val_acc: 0.7766
Epoch 3/30
20/20 [==============================] - 0s 4ms/step - loss: 0.5974 - acc: 0.8020 - val_loss: 0.5679 - val_acc: 0.8503
Epoch 4/30
20/20 [==============================] - 0s 4ms/step - loss: 0.5513 - acc: 0.8498 - val_loss: 0.5248 - val_acc: 0.8807
Epoch 5/30
20/20 [==============================] - 0s 4ms/step - loss: 0.5120 - acc: 0.8805 - val_loss: 0.4875 - val_acc: 0.8915
Epoch 6/30
20/20 [==============================] - 0s 5ms/step - loss: 0.4776 - acc: 0.8861 - val_loss: 0.4546 - val_acc: 0.8959
Epoch 7/30
20/20 [==============================] - 0s 5ms/step - loss: 0.4471 - acc: 0.8892 - val_loss: 0.4253 - val_acc: 0.9067
Epoch 8/30
20/20 [==============================] - 0s 5ms/step - loss: 0.4200 - acc: 0.8894 - val_l

20/20 [==============================] - 1s 41ms/step - loss: 0.6799 - acc: 0.5536 - val_loss: 0.6435 - val_acc: 0.6855
Epoch 2/30
20/20 [==============================] - 0s 4ms/step - loss: 0.6047 - acc: 0.7304 - val_loss: 0.5913 - val_acc: 0.6985
Epoch 3/30
20/20 [==============================] - 0s 4ms/step - loss: 0.5613 - acc: 0.7657 - val_loss: 0.5572 - val_acc: 0.7570
Epoch 4/30
20/20 [==============================] - 0s 4ms/step - loss: 0.5320 - acc: 0.7833 - val_loss: 0.5334 - val_acc: 0.7570
Epoch 5/30
20/20 [==============================] - 0s 4ms/step - loss: 0.5117 - acc: 0.7843 - val_loss: 0.5164 - val_acc: 0.7570
Epoch 6/30
20/20 [==============================] - 0s 4ms/step - loss: 0.4964 - acc: 0.7860 - val_loss: 0.5029 - val_acc: 0.7592
Epoch 7/30
20/20 [==============================] - 0s 4ms/step - loss: 0.4838 - acc: 0.7901 - val_loss: 0.4915 - val_acc: 0.7636
Epoch 8/30
20/20 [==============================] - 0s 4ms/step - loss: 0.4733 - acc: 0.7922 - val_l

20/20 [==============================] - 1s 61ms/step - loss: 0.6920 - acc: 0.6212 - val_loss: 0.6901 - val_acc: 0.6985
Epoch 2/30
20/20 [==============================] - 0s 7ms/step - loss: 0.6872 - acc: 0.7099 - val_loss: 0.6868 - val_acc: 0.7050
Epoch 3/30
20/20 [==============================] - 0s 9ms/step - loss: 0.6838 - acc: 0.7129 - val_loss: 0.6836 - val_acc: 0.7028
Epoch 4/30
20/20 [==============================] - 0s 10ms/step - loss: 0.6804 - acc: 0.7137 - val_loss: 0.6805 - val_acc: 0.7050
Epoch 5/30
20/20 [==============================] - 0s 11ms/step - loss: 0.6771 - acc: 0.7140 - val_loss: 0.6774 - val_acc: 0.7050
Epoch 6/30
20/20 [==============================] - 0s 11ms/step - loss: 0.6738 - acc: 0.7140 - val_loss: 0.6742 - val_acc: 0.7050
Epoch 7/30
20/20 [==============================] - 0s 9ms/step - loss: 0.6705 - acc: 0.7142 - val_loss: 0.6710 - val_acc: 0.7072
Epoch 8/30
20/20 [==============================] - 0s 8ms/step - loss: 0.6670 - acc: 0.7142 - va

20/20 [==============================] - 1s 55ms/step - loss: 0.6798 - acc: 0.4810 - val_loss: 0.6694 - val_acc: 0.4729
Epoch 2/30
20/20 [==============================] - 0s 5ms/step - loss: 0.6634 - acc: 0.4849 - val_loss: 0.6551 - val_acc: 0.4751
Epoch 3/30
20/20 [==============================] - 0s 5ms/step - loss: 0.6509 - acc: 0.7020 - val_loss: 0.6425 - val_acc: 0.9653
Epoch 4/30
20/20 [==============================] - 0s 5ms/step - loss: 0.6377 - acc: 0.9838 - val_loss: 0.6282 - val_acc: 0.9978
Epoch 5/30
20/20 [==============================] - 0s 5ms/step - loss: 0.6229 - acc: 0.9905 - val_loss: 0.6128 - val_acc: 0.9978
Epoch 6/30
20/20 [==============================] - 0s 5ms/step - loss: 0.6073 - acc: 0.9908 - val_loss: 0.5972 - val_acc: 0.9978
Epoch 7/30
20/20 [==============================] - 0s 5ms/step - loss: 0.5918 - acc: 0.9908 - val_loss: 0.5818 - val_acc: 0.9978
Epoch 8/30
20/20 [==============================] - 0s 6ms/step - loss: 0.5765 - acc: 0.9908 - val_l

20/20 [==============================] - 1s 74ms/step - loss: 0.6934 - acc: 0.4599 - val_loss: 0.6870 - val_acc: 0.4295
Epoch 2/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6855 - acc: 0.5168 - val_loss: 0.6804 - val_acc: 0.5835
Epoch 3/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6800 - acc: 0.5889 - val_loss: 0.6769 - val_acc: 0.5770
Epoch 4/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6779 - acc: 0.5877 - val_loss: 0.6757 - val_acc: 0.5770
Epoch 5/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6768 - acc: 0.5877 - val_loss: 0.6744 - val_acc: 0.5770
Epoch 6/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6755 - acc: 0.5877 - val_loss: 0.6726 - val_acc: 0.5770
Epoch 7/30
20/20 [==============================] - 0s 7ms/step - loss: 0.6735 - acc: 0.5877 - val_loss: 0.6698 - val_acc: 0.5770
Epoch 8/30
20/20 [==============================] - 0s 6ms/step - loss: 0.6707 - acc: 0.5877 - val_l

In [9]:
X.shape

(3760, 86)